In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O ./dataset/data.txt

--2024-11-18 16:04:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./dataset/data.txt’

./dataset/data.txt  100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-11-18 16:04:56 (23.5 MB/s) - ‘./dataset/data.txt’ saved [1115394/1115394]



In [2]:
with open('./dataset/data.txt', 'r', encoding='utf-8') as file:
    data = file.read()

In [3]:
print('no. of characters:', len(data))

no. of characters: 1115394


In [4]:
print(data[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [5]:
unique_chars = sorted(list(set(data)))
vocab_size = len(unique_chars)
print('vocab size:', vocab_size)
print('all characters:' + ''.join(unique_chars))

vocab size: 65
all characters:
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [6]:
# create char to index and index to char mapping
char_to_idx = {ch:i for i, ch in enumerate(unique_chars)}
idx_to_char = {i:ch for i, ch in enumerate(unique_chars)}

# tokenizer
encode = lambda s: [char_to_idx[char] for char in s]
decode = lambda l: ''.join([idx_to_char[idx] for idx in l])
print('encoded:', encode('hello'))
print('decoded:', decode(encode('hello')))


encoded: [46, 43, 50, 50, 53]
decoded: hello


In [7]:
import torch
data = torch.tensor(encode(data), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
nm = int(len(data) * 0.9)
train_data = data[:nm]
val_data = data[nm:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print('t:', t, 'context:', context, 'target:', target)

t: 0 context: tensor([18]) target: tensor(47)
t: 1 context: tensor([18, 47]) target: tensor(56)
t: 2 context: tensor([18, 47, 56]) target: tensor(57)
t: 3 context: tensor([18, 47, 56, 57]) target: tensor(58)
t: 4 context: tensor([18, 47, 56, 57, 58]) target: tensor(1)
t: 5 context: tensor([18, 47, 56, 57, 58,  1]) target: tensor(15)
t: 6 context: tensor([18, 47, 56, 57, 58,  1, 15]) target: tensor(47)
t: 7 context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) target: tensor(58)


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split='train'):
    data = train_data if split == 'train' else val_data
    idx = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in idx])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in idx])
    return x, y

xb, yb = get_batch('train')
print(f"input shape{xb.shape}")
print("inputs:")
print(xb)
print(f"target shape{yb.shape}")
print("targets:")
print(yb)
print("=========")

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t + 1]
        target = yb[b, t]
        print('b:', b, 't:', t, 'context:', context, 'target:', target)

input shapetorch.Size([4, 8])
inputs:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target shapetorch.Size([4, 8])
targets:
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
b: 0 t: 0 context: tensor([24]) target: tensor(43)
b: 0 t: 1 context: tensor([24, 43]) target: tensor(58)
b: 0 t: 2 context: tensor([24, 43, 58]) target: tensor(5)
b: 0 t: 3 context: tensor([24, 43, 58,  5]) target: tensor(57)
b: 0 t: 4 context: tensor([24, 43, 58,  5, 57]) target: tensor(1)
b: 0 t: 5 context: tensor([24, 43, 58,  5, 57,  1]) target: tensor(46)
b: 0 t: 6 context: tensor([24, 43, 58,  5, 57,  1, 46]) target: tensor(43)
b: 0 t: 7 context: tensor([24, 43, 58,  5, 57,  1, 46, 43]) target: tensor(39)
b: 1 t: 0 context: tensor([44]) target: tensor(53)
b: 1 t: 1 conte

In [12]:
# sample input for the bigram model
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (Batch, Time, Channels)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T) # (Batch * Time)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx: (B, T) in current context
        for _ in range(max_new_tokens):
            # Get prediction
            logits, loss = self(idx) 
            # Only take the last token
            logits = logits[:, -1, :] 
            # Softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            #  Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append to the sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros(1, 1, dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5727508068084717


In [16]:
print(decode(m.generate(idx = torch.zeros(1, 1, dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


## The maths trick in self-attention

### The more efficient way of using triangluar matrix 

In [17]:
# e.g. token at 4th position take information from previous tokens by taking an average of them

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels, channels is set to 2 for simplicity: curr token infor + avg of prev token info
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [19]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [20]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2,atol=1e-7)

True

In [21]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3, atol=1e-7)

True

In [ ]:
# version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32 
x = torch.randn(B,T,C)

# self-attention explain:
# every token in a single position will emit two vectors: query and key
# query = what token is looking for
# key = what token contains
# dot product of the query to others keys will become that weights
# so that if the query and key are align, which will give a high value, then the learning about that specific token will be high

# single head self-attention
head_size = 16 # H
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False) # the info aggrated for this single head between different nodes
k = key(x) # (B, T, H)
q = query(x) # (B, T, H)
# communication between tokens
wei = q @ k.transpose(-2, -1) # (B, T, H) @ (B, H, T) ---> (B, T, T)

# masking out the upper triangle
tril = torch.tril(torch.ones(T, T))
# Encoder block: mask out the upper triangle, so that hidden states can't see the future
# Decoder block: deleting this line of code will allow all nodes to talk to each others, e.g. sentiment analysis
wei = wei.masked_fill(tril == 0, float('-inf')) 
wei = F.softmax(wei, dim=-1) # normalize the weights

v = value(x) # (B, T, H), v is the element aggregated from the other tokens
out = wei @ v

out.shape


torch.Size([4, 8, 16])

In [33]:
wei[0]
# a look a the last row of the weight matrix
# the last token 0.2391 has a high affinity with 4th token 0.2297 
# thus through the softmax, it can aggregate a lot of info from the 4th token to the last token
# meaning it can learn a lot about it

tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496,    -inf,    -inf,    -inf,    -inf],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,    -inf,    -inf,    -inf],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,    -inf,    -inf],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,    -inf],
        [-1.8044, -0.4126, -0.8306,  0.5898, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below


In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # scaling to control normalization

In [36]:
k.var()

tensor(1.0449)

In [37]:
q.var()

tensor(1.0700)

In [38]:
wei.var()

tensor(1.0918)

In [39]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [42]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])